In [477]:
import pandas as pd

url = "https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/datasets/vgames2.csv"

df = pd.read_csv(url, index_col = 0)
"""
1. 지역에 따라서 선호하는 게임 장르가 다를까 라는 질문에 대답을 하셔야합니다.

2. 연도별 게임의 트렌드가 있을까 라는 질문에 대답을 하셔야합니다.

3. 출고량이 높은 게임에 대한 분석 및 시각화 프로세스가 포함되어야 합니다.
"""   ## ++ 전체적으로 보면 ~나라에서의 판매량이 제일 높다. 그지역을 집중적으로 공략해야 한다. or 특정장르가 꾸준히 인기. or Year에 따라 변화하는 추이에 따라 (Genre, Platform) 설정.
##근데 사실 그림(plot)으로 표현하는게 이해하기에는 최곤데.(https://eda-ai-lab.tistory.com/13) : 이상치 확인 EDA
df

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales
1,Candace Kane's Candy Factory,DS,2008.0,Action,Destineer,0.04,0,0,0
2,The Munchables,Wii,2009.0,Action,Namco Bandai Games,0.17,0,0,0.01
3,Otome wa Oanesama Boku ni Koi Shiteru Portable,PSP,2010.0,Adventure,Alchemist,0,0,0.02,0
4,Deal or No Deal: Special Edition,DS,2010.0,Misc,Zoo Games,0.04,0,0,0
5,Ben 10 Ultimate Alien: Cosmic Destruction,PS3,2010.0,Platform,D3Publisher,0.12,0.09,0,0.04
...,...,...,...,...,...,...,...,...,...
16594,Ice Age 2: The Meltdown,GC,2006.0,Platform,Vivendi Games,0.15,0.04,0,0.01
16595,Rainbow Islands: Revolution,PSP,2005.0,Action,Rising Star Games,0.01,0,0,0
16596,NBA 2K16,PS3,2015.0,Sports,Take-Two Interactive,0.44,0.19,0.03,0.13
16597,Toukiden: The Age of Demons,PSV,2013.0,Action,Tecmo Koei,0.05,0.05,0.25,0.03


In [478]:
# df.EU_Sales.unique()    ##column 값들 대충보기.
#df.info()   ## null data는 불완전 하므로 다 지우기. ('0'이랑 다름. - n133)   
##but Year는 명확하지 않지만 판매량이 엄청난 게임회사가 있을 수도. 확인 필요.Genre도 마찬가지. 진짜 버려도 되는 데이터인지 설멸할 수 있어야 신뢰도 증가.
pd.DataFrame(df.isnull().sum(), columns=["결측치 개수"])

,결측치 개수
Name,0
Platform,0
Year,271
Genre,50
Publisher,58
NA_Sales,0
EU_Sales,0
JP_Sales,0
Other_Sales,0


In [479]:
df = df.dropna(axis=0)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16241 entries, 1 to 16598
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Name         16241 non-null  object 
 1   Platform     16241 non-null  object 
 2   Year         16241 non-null  float64
 3   Genre        16241 non-null  object 
 4   Publisher    16241 non-null  object 
 5   NA_Sales     16241 non-null  object 
 6   EU_Sales     16241 non-null  object 
 7   JP_Sales     16241 non-null  object 
 8   Other_Sales  16241 non-null  object 
dtypes: float64(1), object(8)
memory usage: 1.2+ MB


In [480]:
## K->M으로 단위변환 EDA
dfs = df.iloc[:,5:]   ## 나라별 sales

def toMil(data) :         ##시간날때 df['~'] = df['~'].apply(str)로 더간단화하기.
  if type(data) == str :
    if 'K' in data :
      data= data.replace('K','')
      data = float(data)*10**(-3)
    elif 'M' in data :
      data = data.replace('M','')
    else : data = data      ## 숫자형 data와 str형 data가 섞여있나..? (str제거 한다음 dtype 확인해보면 정확.) - 별로안중요.
  return data

for i in range(len(dfs.columns)):
  dfs.iloc[:,i] = dfs.iloc[:,i].apply(toMil)
  dfs.iloc[:,i] = dfs.iloc[:,i].apply(pd.to_numeric)

dfs

,NA_Sales,EU_Sales,JP_Sales,Other_Sales
1,0.04,0.00,0.00,0.00
2,0.17,0.00,0.00,0.01
3,0.00,0.00,0.02,0.00
4,0.04,0.00,0.00,0.00
5,0.12,0.09,0.00,0.04
...,...,...,...,...
16594,0.15,0.04,0.00,0.01
16595,0.01,0.00,0.00,0.00
16596,0.44,0.19,0.03,0.13
16597,0.05,0.05,0.25,0.03


In [481]:
df1 = df.iloc[:,0:4]
df1
df = pd.concat([df1,dfs],axis=1)    ## 합치기, merge로도 해보기.

dfg = df.groupby('Genre')[['NA_Sales','EU_Sales','JP_Sales','Other_Sales']].sum()
dfg    ##value를 수치화해서 % barplot subplot으로 큰 순서대로 그려넣으면 좋을듯. ##약간의 공통된 경향이 있긴 한데 그걸 어떻게 수치화해서 보여줄지.

,NA_Sales,EU_Sales,JP_Sales,Other_Sales
Genre,,,,
Action,858.75,513.64,156.60,183.99
Adventure,101.79,63.66,51.87,16.70
Fighting,220.35,99.90,87.14,36.15
Misc,394.73,208.15,104.18,72.64
Platform,445.38,200.63,130.53,51.45
Puzzle,121.58,50.47,56.24,12.44
Racing,354.70,235.02,56.32,76.51
Role-Playing,318.76,183.09,342.93,56.73
Shooter,574.48,310.23,38.18,101.86


In [482]:
## Year EDA
# df.Year.unique()    ## 발표자료에 넣기. ~한 값들이 있더라. 네모로 해서. 전처리가 필요하다 의 근거. (96, 9, 6, 16,,,->19__ or 20__)

# df.dtypes   ## Year가 float형태.

def toYear(y) :
  if y < 10 :
    y = str(y)
    y = '200'+y
  elif (y>=10) & (y<50):
    y = str(y)
    y = '20'+y
  elif (y>=50) & (y<1000):
    y = str(y)
    y = '19'+y
  else : y = y
  return y

df['Year'] = df['Year'].apply(toYear)
# df['Year'].unique()     ## 확인.


dfy = df.groupby('Year')[['NA_Sales','EU_Sales','JP_Sales','Other_Sales']].sum()
dfy     ## year별로 각나라 전체 sales 추이 line 그래프로.

,NA_Sales,EU_Sales,JP_Sales,Other_Sales
Year,,,,
1980.0,10.13,0.64,0.00,0.11
1981.0,33.40,1.96,0.00,0.32
1982.0,26.92,1.65,0.00,0.31
1983.0,7.76,0.80,8.10,0.14
1984.0,33.28,2.10,14.27,0.70
...,...,...,...,...
2012.0,0.29,0.35,0.02,0.10
2013.0,0.00,0.00,0.21,0.00
2014.0,1.46,0.54,0.39,0.37


In [483]:
## 출고량이 제일 높은 게임에 대한 분석  ##1. genre, 2.Name
# dfg['sum'] = dfg[,:].sum()
dfg['sum'] = dfg.sum(axis=1)
dfg = dfg.sort_values(by='sum' ,ascending=False)
dfg     ##전체 출고량을 보면 Action > Sports > Shooter가 인기 있다. (%, 분산에 따라..?)

,NA_Sales,EU_Sales,JP_Sales,Other_Sales,sum
Genre,,,,,
Action,858.75,513.64,156.60,183.99,1712.98
Sports,667.26,370.97,134.76,132.35,1305.34
Shooter,574.48,310.23,38.18,101.86,1024.75
Role-Playing,318.76,183.09,342.93,56.73,901.51
Platform,445.38,200.63,130.53,51.45,827.99
Misc,394.73,208.15,104.18,72.64,779.70
Racing,354.70,235.02,56.32,76.51,722.55
Fighting,220.35,99.90,87.14,36.15,443.54
Simulation,181.43,112.99,63.54,31.32,389.28


In [484]:
## 역대 가장 인기 있는 game
dfs['sum'] = dfs.sum(axis=1)

df2 = pd.concat([df1,dfs],axis=1)    ## df1 : sales제외한 df의 앞 column부분,   dfs : df의 sales부분
df2 = df2.sort_values(by='sum' ,ascending=False)      ##intro로 이렇게 격차가 크다(출시게임을 설계하는게 중요하다 & 관심끌기)는 걸 보여주면 좋을듯.(++ 특정지역에서만 성공? or 한지역에서의 성공이 다른 지역에서도 성공을 보장하는지. 관련성파악하기.)
df2[0:10]

,Name,Platform,Year,Genre,NA_Sales,EU_Sales,JP_Sales,Other_Sales,sum
2909,Wii Sports,Wii,2006.0,Sports,41.49,29.02,3.77,8.46,82.74
4200,Super Mario Bros.,NES,1985.0,Platform,29.08,3.58,6.81,0.77,40.24
12580,Mario Kart Wii,Wii,2008.0,Racing,15.85,12.88,3.79,3.31,35.83
3353,Wii Sports Resort,Wii,2009.0,Sports,15.75,11.01,3.28,2.96,33.00
7830,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,11.27,8.89,10.22,1.00,31.38
1204,Tetris,GB,1989.0,Puzzle,23.20,2.26,4.22,0.58,30.26
6938,New Super Mario Bros.,DS,2006.0,Platform,11.38,9.23,6.50,2.90,30.01
3015,Wii Play,Wii,2006.0,Misc,14.03,9.20,2.93,2.85,29.01
16532,New Super Mario Bros. Wii,Wii,2009.0,Platform,14.59,7.06,4.70,2.26,28.61
10489,Duck Hunt,NES,1984.0,Shooter,26.93,0.63,0.28,0.47,28.31
